In [1]:
import re
import glob
import feather
import itertools
import numpy as np
import pandas as pd
import unicodedata
import matplotlib.pyplot as plt
from pathlib import Path

# dask
import dask
import dask.dataframe as dd
from dask.distributed import Client
from dask.delayed import delayed


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline
%config IPCompleter.use_jedi = False

## Dask

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:38931 Dashboard: http://127.0.0.1:8787,Cluster Workers: 8 Cores: 8 Memory: 10.02 GB


In [4]:
participantes_raw = Path('../data/raw/participantes/').resolve()
participantes_processed = Path('../data/processed/participantes/').resolve()
files_raw = []
files_processed = []

for file in participantes_raw.glob('participantes_*'):
    files_raw.append(file.as_posix())
    files_processed.append((participantes_processed / file.name.split('.csv')[0]).as_posix())

In [5]:
dfs = [delayed(clean_file, pure=True)(file) for file in files_raw]
writes = [delayed(write_file)(df, fn) for df, fn in zip(dfs, files_processed)]
dask.compute(*writes)

(None, None, None, None)

## Limpiar la clave de una unidad compradora

In [6]:
df_participantes_2012_2013 = pd.read_csv(
    '../data/processed/participantes/participantes_2012_2013.psv', sep='|',
    dtype={
        'TIPO_PROCEDIMIENTO': 'category', 'TIPO_CONTRATACION': 'category', 'FORMA_PROCEDIMIENTO': 'category',
        'ESTATUS_DE_PROPUESTA': 'category', 'ESTATUS_PARTIDA': 'category', 'ESTATUS_FALLO': 'category',
        'PERIODO': 'category', 'DESCRIPCION_CUCOP': str, 'PROVEEDOR_CONTRATISTA': str, 'UNIDAD_DE_MEDIDA': str,
        'EXPEDIENTE': str
    }
)

In [7]:
df_participantes_2012_2013.loc[df_participantes_2012_2013.CLAVEUC.str.len() == 27].NOMBRE_DE_LA_UC.values

array([ 'IMSS-OBRA DE LA UMAE HOSPITAL DE CARDIOLOGIA #34 EN MONTERREY NUEVO LEON #019GYR995'], dtype=object)

In [8]:
df_participantes_2012_2013.loc[df_participantes_2012_2013.CLAVEUC.str.len() == 27, 'CLAVEUC'] = '019GYR995'

In [9]:
df_participantes_2012_2013.CLAVEUC.str.len().value_counts()

9.0    1817415
Name: CLAVEUC, dtype: int64

In [10]:
df_participantes_2012_2013.to_csv(
    '../data/processed/participantes/participantes_2012_2013.psv', index=False, quoting=1, encoding='utf-8', sep='|'
)

## End of the cleaning

In [2]:

# df_participantes = dd.from_delayed(dfs)

# CLAVEUC = df_participantes.NOMBRE_DE_LA_UC.map(get_claveuc_nombre)
# df_participantes = df_participantes.assign(CLAVEUC=CLAVEUC)
# df_participantes.head()

In [3]:
# df_test = pd.read_csv(
#     '../data/raw/participantes/participantes_2010-2011.csv', encoding='iso-8859-1', dtype={'Expediente': str})
# print(df_test.columns)

In [22]:
# df_salida = df_test.loc[
#     :, ['ESTATUS_DE_PROPUESTA', 'ESTATUS_FALLO', 'ESTATUS_PARTIDA', 'NUMERO_PROCEDIMIENTO']
# ].groupby(['ESTATUS_DE_PROPUESTA', 'ESTATUS_FALLO', 'ESTATUS_PARTIDA']).NUMERO_PROCEDIMIENTO.count()
# df_salida = df_salida.reset_index()
# df_salida.sort_values('NUMERO_PROCEDIMIENTO').to_csv(
#     '../data/processed/estatus_2016_2017.csv', index=False, quoting=1, encoding='utf-8'
# )
# df_salida.sort_values('NUMERO_PROCEDIMIENTO', ascending=False)

In [5]:
# df_participantes.to_csv(
#     '../data/processed/participantes_all_2017_09_13.psv', encoding='utf-8', quoting=1, sep='|', index=False)

In [2]:
df_participantes = pd.read_csv(
    '../data/processed/participantes_all_2017_09_13.psv', sep='|',
    dtype={
        'TIPO_PROCEDIMIENTO': 'category', 'TIPO_CONTRATACION': 'category', 'FORMA_PROCEDIMIENTO': 'category',
        'ESTATUS_DE_PROPUESTA': 'category', 'ESTATUS_PARTIDA': 'category', 'ESTATUS_FALLO': 'category',
        'PERIODO': 'category'
    }
)
df_participantes.head()

,NUMERO_PROCEDIMIENTO,DEPENDENCIA,NOMBRE_DE_LA_UC,TIPO_PROCEDIMIENTO,CARACTER,TIPO_CONTRATACION,FORMA_PROCEDIMIENTO,NOMBRE_DE_LICITANTE,ESTATUS_DE_PROPUESTA,CLAVE_CUCOP,DESCRIPCION_CUCOP,CANTIDAD,UNIDAD_DE_MEDIDA,PRECIO_UNITARIO,ESTATUS_PARTIDA,PRECIO_TOTAL,ESTATUS_FALLO,EXPEDIENTE,PROJECT_TYPE,PROJECT_SHORT_DESCRIPTION,PERIODO,CLAVEUC
0,AA-019GYR071-E350-2015,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS-HOSPITAL DE TRAUMATOLOGIA Y ORTOPEDIA LOM...,ADJUDICACION DIRECTA,NACIONAL,ADQUISICIONES,PRESENCIAL,LUCRECIA MONTOYA ROJAS,PERDEDOR,25400519.0,TUBO ENDOTRAQUEAL SIN GLOBO DE ELASTOMERO DE S...,48.0,PIEZA,35.00,NO ADJUDICADO,1680.00,GANADOR,1015530.0,05. ADJUDICACION DIRECTA LAASSP,"TUBOS ENDOTRAQUEALES, SIN GLOBO.",2016-2017,019GYR071
1,AA-019GYR071-E350-2015,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS-HOSPITAL DE TRAUMATOLOGIA Y ORTOPEDIA LOM...,ADJUDICACION DIRECTA,NACIONAL,ADQUISICIONES,PRESENCIAL,FARMADESCUENTO,GANADOR,25400519.0,TUBO ENDOTRAQUEAL SIN GLOBO DE ELASTOMERO DE S...,48.0,PIEZA,20.00,NO ADJUDICADO,960.00,GANADOR,1015530.0,05. ADJUDICACION DIRECTA LAASSP,"TUBOS ENDOTRAQUEALES, SIN GLOBO.",2016-2017,019GYR071
2,AA-019GYR071-E350-2015,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS-HOSPITAL DE TRAUMATOLOGIA Y ORTOPEDIA LOM...,ADJUDICACION DIRECTA,NACIONAL,ADQUISICIONES,PRESENCIAL,FARMADESCUENTO,GANADOR,25400519.0,TUBO ENDOTRAQUEAL SIN GLOBO DE ELASTOMERO DE S...,72.0,PIEZA,18.00,NO ADJUDICADO,1296.00,GANADOR,1015530.0,05. ADJUDICACION DIRECTA LAASSP,"TUBOS ENDOTRAQUEALES, SIN GLOBO.",2016-2017,019GYR071
3,AA-019GYR071-E350-2015,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS-HOSPITAL DE TRAUMATOLOGIA Y ORTOPEDIA LOM...,ADJUDICACION DIRECTA,NACIONAL,ADQUISICIONES,PRESENCIAL,"TECNOLOGIA MEDICA DIART, S.A. DE .C.V.",PERDEDOR,25400519.0,TUBO ENDOTRAQUEAL SIN GLOBO DE ELASTOMERO DE S...,48.0,PIEZA,37.13,NO ADJUDICADO,1782.24,GANADOR,1015530.0,05. ADJUDICACION DIRECTA LAASSP,"TUBOS ENDOTRAQUEALES, SIN GLOBO.",2016-2017,019GYR071
4,AA-019GYR071-E350-2015,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS-HOSPITAL DE TRAUMATOLOGIA Y ORTOPEDIA LOM...,ADJUDICACION DIRECTA,NACIONAL,ADQUISICIONES,PRESENCIAL,"TECNOLOGIA MEDICA DIART, S.A. DE .C.V.",PERDEDOR,25400519.0,TUBO ENDOTRAQUEAL SIN GLOBO DE ELASTOMERO DE S...,72.0,PIEZA,37.13,NO ADJUDICADO,2673.36,GANADOR,1015530.0,05. ADJUDICACION DIRECTA LAASSP,"TUBOS ENDOTRAQUEALES, SIN GLOBO.",2016-2017,019GYR071


In [6]:
df_test = df_participantes.loc[~df_participantes.NOMBRE_DE_LA_UC.isnull()]
df_test = df_test.loc[df_test.NOMBRE_DE_LA_UC.str.contains('Recursos Materiales y servicios'.upper())]

In [8]:
df_test.loc[df_test.PROJECT_SHORT_DESCRIPTION.str.contains('DATOS ABIERTOS')]

,NUMERO_PROCEDIMIENTO,DEPENDENCIA,NOMBRE_DE_LA_UC,TIPO_PROCEDIMIENTO,CARACTER,TIPO_CONTRATACION,FORMA_PROCEDIMIENTO,NOMBRE_DE_LICITANTE,ESTATUS_DE_PROPUESTA,CLAVE_CUCOP,DESCRIPCION_CUCOP,CANTIDAD,UNIDAD_DE_MEDIDA,PRECIO_UNITARIO,ESTATUS_PARTIDA,PRECIO_TOTAL,ESTATUS_FALLO,EXPEDIENTE,PROJECT_TYPE,PROJECT_SHORT_DESCRIPTION,PERIODO,CLAVEUC


In [2]:
df = feather.read_dataframe('../data/processed/procedimientos_all_2017_07_19.feather')
df.head()

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,CLAVEUC_REAL,IMPORTE_PESOS
0,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,RAUL EUGENIO ALVAREZ CARRASCO,HABILITADO,MICRO,512930.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS 3 PERSONAS,208993,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-12-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,None,No,MX,NaN,None,NaN,901024986,512930.40
1,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,MA DE LA LUZ TISCARENO ELIAS,HABILITADO,NO MIPYME,1887409.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS 3 PERSONAS,208992,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-10-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,None,No,MX,NaN,None,NaN,901024986,1887409.40
2,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,PAPELERIA CONSUMIBLES Y ACCESORIOS SA DE CV,HABILITADO,MICRO,1892535.82,NaN,MXN,IA-901024986-N53-2012,None,INVITACION A CUANDO MENOS 3 PERSONAS,257525,IEA.DJ.CONT. 1140/2012,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-11-05 14:34:00,2012-11-12 10:00:00,2012-10-19,NaT,2012-11-12,2012-12-31,268049,IEA PAPELERIA Y CONSUMIBLES,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,None,No,MX,NaN,None,NaN,901024986,1892535.82
3,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS SA DE CV,HABILITADO,PEQUENA,24234.00,NaN,MXN,AA-901024986-N31-2012,None,ADJUDICACION DIRECTA FEDERAL,241054,CONSUMIBLES,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-09-06 08:49:00,2012-09-07 09:00:00,NaT,NaT,2012-09-11,2012-12-31,252725,COORDINACION ESTATAL DE ASESORIA Y SEGUIMIENTO...,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,None,No,MX,NaN,None,NaN,901024986,24234.00
4,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS SA DE CV,HABILITADO,PEQUENA,18700.00,NaN,MXN,AA-901024986-N64-2012,None,ADJUDICACION DIRECTA FEDERAL,474294,ADQUISICION DE EQUIPO DE COMPUTO,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-12-12 11:38:00,2012-12-14 10:00:00,NaT,NaT,2012-12-17,2012-12-31,320854,EQUIPO DE COMPUTO DESARROLLO EDUCATIVO,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,None,No,MX,NaN,None,NaN,901024986,18700.00


In [ ]:
# files_raw, files_processed = tuple(
#     zip(*[(file.as_posix(), (participantes_processed / file.name).as_posix())
#           for file in participantes_raw.glob('participantes_*')])
# )

# df_test = load_data('../data/raw/perdedores/participantes_2010-2011.csv')
# df_test = load_data('../data/raw/perdedores/participantes_2014_2015.csv')
# df_test = load_data('../data/raw/perdedores/participantes_2016_2017.csv')
# df_test.head()

# df_participantes = delayed(pd.concat)(dfs, axis=0, ignore_index=True)

# df_participantes = df_participantes.compute()

In [ ]:
# def clean_participantes_file(path: str):
#     df = pd.read_csv(path, encoding='iso-8859-1', dtype={'Expediente': str})
    
#     # Add the period according to the file name
#     periodo = path.split('.csv')[0]
#     periodo = '-'.join(find_numbers(periodo))
#     df = df.assign(periodo=periodo)
    
#     # Normalize column names
#     df = df.rename(
#         columns={
#             'Dependencia/Entidad': 'DEPENDENCIA', 'Nº de Procedimiento': 'NUMERO_PROCEDIMIENTO',
#             'Unidad Compradora': 'NOMBRE_DE_LA_UC', 'Forma del procedimiento': 'FORMA_PROCEDIMIENTO',
#             'Nombre de Licitante': 'PROVEEDOR_CONTRATISTA', 'Expediente': 'CODIGO_EXPEDIENTE'
#         }
#     )
#     new_columns = {
#         col: unicodedata.normalize('NFD', col).encode('ascii', 'ignore').decode('utf-8')
#         for col in df.columns
#     }
#     new_columns = {
#         k: '_'.join(v.split(' ')).upper()
#         for k, v in new_columns.items()
#     }
#     df = df.rename(columns=new_columns)
    
#     # Normalize string column values
#     cols = {k for k, v in df.dtypes.to_dict().items() if str(v) == 'object'}
#     for col in cols:
#         df.loc[:, col] = df[col].str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.upper()
#         df.loc[:, col] = df[col].str.replace('.', '')
#         df.loc[:, col] = df[col].str.replace(',', '')
#         df.loc[:, col] = df[col].str.strip()
       
#     # Extract clave from UC's names
#     CLAVEUC = df.NOMBRE_DE_LA_UC.map(get_claveuc_nombre)
#     df = df.assign(CLAVEUC=CLAVEUC)
    
#     # Save as category to reduce memory usage
#     cols = (
#         'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION', 'FORMA_PROCEDIMIENTO', 'PERIODO',
#         'ESTATUS_PARTIDA', 'ESTATUS_FALLO', 'ESTATUS_DE_PROPUESTA', 'DEPENDENCIA',
#         'CLAVEUC', 'PERIODO'
#     )
#     for col in cols:
#         df.loc[:, col] = df[col].astype('category')
        
#     # Remove common endings
#     regex_list = [
#         '[S]\s?[A]\s+[D]\s?[E]\s+[C]\s?[V]', # SA DE CV
#         '[S]\s?[A]\s?[P]\s?[I]\s+[D]\s?[E]\s+[C]\s?[V]', # SAPI DE CV
#         '[S]\s?[A]\s?[B]\s+[D]\s?[E]\s+[C]\s?[V]',  # SAB DE CV
#         '[S]\s?[D]\s?[E]\s+[R]\s?[L]\s+[D]\s?[E]\s+[C]\s?[V]', # S DE RL DE CV
#         '[S]\s?[C]\s+[D]\s?[E]\s+[R]\s?[L]\s+[D]\s?[E]\s+[C]\s?[V]', # SC DE RL DE CV
#         '[S]\s?[C]\s?[P]\s+[D]\s?[E]\s+[R]\s?[L]\s+[D]\s?[E]\s+[C]\s?[V]', # SCP DE RL DE CV
#         '[S]\s?[P]\s?[R]\s+[D]\s?[E]\s+[R]\s?[L]\s+[D]\s?[E]\s+[C]\s?[V]', # SPR DE RL DE CV
#         '[P]\s?[R]\s+[D]\s?[E]\s+[R]\s?[L]\s+[D]\s?[E]\s+[C]\s?[V]', # PR DE RL DE CV
#         '[S]\s?[C]\s+[D]\s?[E]\s+[C]\s+[D]\s?[E]\s+[R]\s?[L]\s+[D]\s?[E]\s+[C]\s?[V]', # SC DE C DE RL DE CV
#     ]
#     for regex in regex_list:
#         pattern = re.compile(regex)
#         df = df.assign(
#             PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.map(lambda string: remove_pattern(string, pattern))
#         )
#     df = df.assign(PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.str.strip())
#     return df